Data Partition as Training and Test sets

In [1]:
import os, sys
import image_norm_test as myData;
import numpy as np
from sklearn.cross_validation import train_test_split
seed = 3071986   # set random seed
rng = np.random.RandomState(seed)

# fetch all the file in the directory path  defined#
pwd = os.pardir;
accessPath = r"%s\patch_database" %pwd;
filelist = []
print(accessPath)
def fetchFile(accessPath , numOfimageOfeachSample = 91):
    filelist = []
    index = 0
    i = 0
    for file in os.listdir(accessPath):
        if(index % 92 == i):
            completeName = os.path.join(accessPath, file)  
            filelist.append(completeName)
            if(i < numOfimageOfeachSample-1):  
                i+=1
            else:
                i = 0
        index +=1
    return filelist

# in this way we can extract only certain number image of each samples, ex. 2 means only 2 images of one texture
filelist = fetchFile(accessPath , 4)
#print(filelist)


..\patch_database


In [2]:
from sklearn import preprocessing

# will load data as the patch size defined , 3 means 3*3 = 9 for each patch, and will return the dictionary included:
# 'data'  (one patch)  , 'target' (the sample of this patch belongs to ) , 'filename' (the file comes from)
mydata = myData.load_data(filelist[0:25] , 3)
scaler = preprocessing.StandardScaler()

#print(len(mydata))
#print(mydata.keys())


X, y = mydata['data'], mydata['target']
#print(len(X))
#print(X[0])
#print(X[0])

# Normalized all the data that we are going to use
x_norm = scaler.fit_transform(X)
#print(mydata['filename'])
#print(y)

# split our data into half of train data and half of text data randomly.
train_X, test_X, train_y, test_y = train_test_split(x_norm, y, train_size=0.5, random_state=seed)


#print(train_y)
#print(test_y)
#print(test)


Start to get the features learned from feed in data

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import learnvocabulary as LV

'''
start_time = time.time()
#print(train_X[0])
#print(len(train_X)*0.5)  #100000.0
num_data = 10000
clusters = [2,16,32,64,128,256]
for cluster in clusters:
    start_time = time.time()
    centroid = LV.learnvocabulary(train_X, num_data ,rng , cluster_num = cluster , T = 50)

    print("--- %s seconds ---" % (time.time() - start_time))

    centroid_file = r"%s\%dfeature_%d.txt" %(pwd ,  cluster , num_data);
    f = open(centroid_file, 'w')
    for items in centroid:
        for item in items:
            f.write("%s\n" % item  )
    f.close()
#print(centroid)
'''
def read_features(file_name):
    features = list()

    f = open(file_name)
    for line in f.readlines():
        feature_list = line.split(" ")
        feature = list()
        for item in feature_list:
            if item != '\n':
                feature.append(float(item))
        features.append(feature)

    f.close()
    return features


Retrive centroids ( features ) from file stored in disk

In [69]:
centroid = []
count = 0
temp = []
n = 3*3
num_data = 10000
clusters = [16,32,64,128]
#cluster = 16
#centroid_file = r"%s\%dfeature_%d.txt" %(pwd ,  cluster , num_data);
for cluster in clusters:
    centroid_file = r"%s\25_4_3_%d.txt" %(pwd , cluster)
    centroid.append(read_features(centroid_file))
'''
data = np.loadtxt(centroid_file,delimiter="\n")
for i in range(0 , len(data)):
    temp.append(data[i])
    count += 1
    if(count == n):
        centroid.append(temp)
        temp = []
        count = 0
'''
#print(centroid)
#print(len(centroid))

'\ndata = np.loadtxt(centroid_file,delimiter="\n")\nfor i in range(0 , len(data)):\n    temp.append(data[i])\n    count += 1\n    if(count == n):\n        centroid.append(temp)\n        temp = []\n        count = 0\n'

Get the bag of features for each patch 

In [85]:
import bagOfFeature
bofs = []
dot = bagOfFeature.getbofs(centroid[0])
for data in train_X[0:10000]:
    bofs.append(dot.getbof(data) )
print(bofs[0])
#print(X_train_tf)
#print(train_y)

[4.715065569942035, -4.7515505539290066, -2.8952694109471411, -0.50492700395561485, -4.6420701326577714, -3.6608258125979916, 1.9651125215511773, -4.2568858349784309, -4.6463117799052833, -4.4033474297375799, 4.3624538635138723, -2.5407944422441391, -3.6599234548550377, -3.9295758218665875, -0.95387961022450596, 4.7359841337155419]


divide the number of occurrences of each word in a document by the total number of words in the document
(Term Frequency times Inverse Document Frequency)

In [86]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(bofs)
X_train_tf = tf_transformer.transform(bofs)

Used the bag of features of each patch and the known lable to do classification with knn 

In [87]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import grid_search

#knn_init = KNeighborsClassifier()
#parameters = {'n_neighbors':[ 5, 10 , 15]}
#knn = grid_search.GridSearchCV(knn_init, parameters)
knn = KNeighborsClassifier(n_neighbors = 64)

knn.fit(X_train_tf, train_y[0:10000])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=64, p=2,
           weights='uniform')

In [88]:
bofs_test = []
for data in test_X[0:5000]:
    bofs_test.append(dot.getbof(data) )
#print(bofs)
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(bofs_test)
X_test_tf = tf_transformer.transform(bofs_test)


In [89]:
import numpy as np
#predicted = knn.predict(X_test_tf)
knn.score(X_test_tf,test_y[0:5000])

 
#np.mean(predicted == test_y)  

0.25519999999999998